In [15]:
import random
import numpy as np
import copy
import mnist
import matplotlib.pyplot as plt
import time
import copy
import pickle

In [16]:
class Kmeans ():
    
    def __init__(self,org_data,k,T):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
    
    
    def fit(self,data):
        self.data = data
        
        self.initialize()
        self.lloyds()
        #return np.array(fin_centroids),fin_clusters,loss
    
    
    def initialize(self):
    
        n = self.data.shape[0]
        d = self.data.shape[1]
        
        centroid = random.choice(self.data)
        self.centroids = np.array([centroid])

        for i in range(self.k-1):
            Probs = self.get_probs()
            idx = np.random.choice(n,p=Probs)
            centroid = self.data[idx]
            self.centroids = np.vstack([self.centroids,centroid])

        #return init_centroids
    
    def get_probs(self):
    
        Probs = np.zeros([len(self.data)])
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            Probs[i] = np.min(dis)
        Probs = [dist**2 for dist in Probs]
        Probs = Probs/sum(Probs)
        return Probs
    
    
    def lloyds(self):
        T = self.T
        
        for it in range(self.T):
            self.find_clusters()
            self.find_centroids()
            
        #return centroids,clusters,loss
    
    
    def find_clusters(self):
        
        self.loss = 0
        self.clusters = {j:[] for j in range(self.k) }
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            self.loss = self.loss + (np.min(dis))**2
            j = np.argmin(dis)
            self.clusters[j].append(i)
        
        #return clusters,loss
    
    
    def find_centroids(self):
        
        self.centroids = []
        for j in self.clusters.keys():
            if(len(self.clusters[j]) != 0):
                runn_sum = np.zeros(len(self.data[0]))
                count = 0
                for itm in self.clusters[j]: 
                    runn_sum  = runn_sum + self.data[itm]
                    count +=1
                centroid = runn_sum/count
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])
            else:
                centroid = self.reassign_cluster()
                #print("type",type(self.centroids))
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])

        self.centroids = np.array(self.centroids)
    
    def reassign_cluster(self):
        
        #print("reassign_cluster",centroids.shape)
        
        self.centroids = np.array(self.centroids)
        
        if(len(self.centroids) == 0):
            centroid = random.choice(self.data)
        else:
            Probs = self.get_probs()
            idx = np.random.choice(len(self.data),p=Probs)
            centroid = self.data[idx]

        return centroid
    
    
    def delete(self,idx):
        self.dels.append(idx)
        new_data = np.delete(self.org_data,self.dels,axis=0)
        self.fit(new_data)
    
    

In [17]:
class metadata:
    
    def __init__(self,centroids,q_centroids,phase,cluster_sizes):
        self.centroids = centroids
        self.q_centroids = q_centroids
        self.phase = phase
        self.cluster_sizes = cluster_sizes
        
class Q_Kmeans(Kmeans):
    
    def __init__(self,org_data,k,T,gamma,epsilon):
        Kmeans.__init__(self,org_data,k,T)
        self.gamma = gamma
        self.epsilon = epsilon
        self.metadata = None
        self.initial_centroids = None
        
    
    def qfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.metadata = []
        
        self.initialize()
        self.initial_centroids = copy.deepcopy(self.centroids)
        
        self.find_clusters()
        cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
        m0 = metadata(self.initial_centroids,self.initial_centroids,0,cluster_sizes)
        self.metadata.append(m0)
        
        for tau in range(self.T):
            prev_centroids = copy.deepcopy(self.centroids)
            self.find_centroids()
            
            self.imbalance_correction(prev_centroids)
            
            theta = np.random.random([d])
            centroids_hat = self.quantize(self.centroids,theta)
            
            # stored actual self values here
            actual_centroids = copy.deepcopy(self.centroids)
            actual_clusters = copy.deepcopy(self.clusters)
            actual_loss = self.loss
            
            # set centroids_hat as self.centroids
            self.centroids = centroids_hat
            self.find_clusters()
            
            cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
            m = metadata(actual_centroids,self.centroids,theta,cluster_sizes)
            self.metadata.append(m)
            
            if(self.loss>=actual_loss):
                self.centroids = actual_centroids
                self.clusters = actual_clusters
                self.loss = actual_loss
                
                break
                
        
    def imbalance_correction(self,prev_centroids):
        n = len(self.data)
        
        for kappa in range(1,self.k+1):
            val1 = len(self.clusters[kappa-1])
            val2 = self.gamma*n/self.k
            if(val1 < val2):
                self.centroids[kappa-1] = (val1/val2)*self.centroids[kappa-1] + ((val2 - val1)/val2)*prev_centroids[kappa-1] 
    
    def quantize(self,c,theta):
        epsilon = self.epsilon
        c = c* 1/epsilon
        c = c + theta - 0.5
        c = np.round(c)
        c = c - theta + 0.5
        c = c * epsilon
        return c
    
    
    def qdelete(self,idx):
        self.dels.append(idx)
        n = len(self.data)
        
        p = self.org_data[idx]
        
        if p in self.initial_centroids:
            new_data = np.delete(self.org_data,self.dels,axis=0)
            self.qfit(new_data)
            
        else:
            
            flag = 1
            for tau in range(len(self.metadata)-1):
    
                state = self.metadata[tau+1]
                c = state.centroids
                theta = state.phase
                cluster_sizes = state.cluster_sizes
                
                #p belongs to which cluster
                kappa = self.which_cluster(p,c)
                ck = c[kappa]
                
                #c_prevk
                prev_state = self.metadata[tau]
                c_prev = prev_state.q_centroids
                c_prevk = c_prev[kappa]
                
                #clustersize
                val1 = cluster_sizes[kappa] 
                
                # gamma imbalance check
                val2 = self.gamma*n/self.k
                if(val1 < val2):
                    lag = (val2 - val1)/val2
                    ck = (val1/val2)*ck
                    ck = ck + lag*c_prevk
                 
                
                #perturbing center by removing point p
                perturbed_ck = ck - p/val2
                
                #quantize
                cq = self.quantize(ck,theta)
                perturbed_cq = self.quantize(perturbed_ck,theta)
                c_prev[kappa] = perturbed_cq
                

                if not all(cq == perturbed_cq):
                    print("retrain")
                    flag = 0
                    new_data = np.delete(self.org_data,self.dels,axis=0)
                    self.qfit(new_data)
                    break
                else:
                    state.cluster_sizes[kappa] = state.cluster_sizes[kappa]-1
                    new_state = metadata(c,c_prev,theta,state.cluster_sizes)
                    self.metadata[tau+1] = new_state
                    self.data = np.delete(self.org_data,self.dels,axis=0)
              
            if (flag):
                print("no retrain")
                
            print(idx,len(self.data))
            print()
            print()
                    
    def which_cluster(self,p,centroids):
        dis = np.linalg.norm(p - centroids,axis=1)
        kappa = np.argmin(dis)
        return kappa
        

In [18]:

class node:
    '''
    Class for a node of the tree used in the DC tree
    '''
    
    def __init__(self,lvl,parent=None):
        self.children = None
        self.lvl = lvl
        self.dataset = []
        self.centroids = []
        self.parent = parent
        
            
class tree:
    '''
    Class for the DC tree
    '''
    
    def __init__(self,w,h):
        '''
        Sets the tree parameters
        w: # of children for each non leaf node
        h: height of the DC tree
        '''
        self.w = w
        self.h = h
        self.levels = {i:[] for i in range(h)}
        self.root = None
        self.leaves = []
        self.which_leaf = None
        
        
    def initialize(self):
        '''
        Constructs the tree with tree parameters
        '''
        self.root = node(0)
        self.make_tree(self.root,self.w,self.h)
        self.levels[0]=[self.root]
        
        
    def make_tree(self,root,w,h):
        if(h>1):
            root.children = [ node(root.lvl+1,root) for i in range(w)]
            self.levels[root.lvl+1] = self.levels[root.lvl+1] + root.children
            if(h>2):
                for i in range(w):
                    self.make_tree(root.children[i],w,h-1)
            elif(h==2):
                self.leaves = self.leaves+root.children
            
        
    def lvl_traverse(self):
        for i in range(len(self.levels)):
            print(i,len(self.levels[i]))
            

##################################################################################


class DC_Kmeans():
    
    def __init__(self,org_data,k,T,w,h):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
        self.w = w
        self.h = h
        self.tree = None
        
        
    def dcfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.tree = tree(self.w,self.h+1)
        self.tree.initialize()
        
        self.tree.which_leaf = {i:0 for i in range(n)}
        
        for i in range(n):
            leaf = random.choice(self.tree.leaves)
            leaf.dataset.append(np.array(self.data[i]))
            self.tree.which_leaf[i]=leaf
            
        for l in range(self.h,-1,-1):
            for node in self.tree.levels[l]:
                km = Kmeans(np.array(node.dataset),self.k,self.T)
                km.fit(np.array(node.dataset))
                c,cl,loss = km.centroids,km.clusters,km.loss
                node.centroids = c
                if l>0:
                    node.parent.dataset.extend(c)
                else:
                    ## l = 0
                    kmf = Kmeans(self.data,self.k,self.T)
                    kmf.centroids,kmf.data = c,self.data
                    kmf.find_clusters()
                    
                    self.centroids = c
                    self.loss = kmf.loss
                    self.clusters = kmf.clusters
                    
                    
    def dcdelete(self,idx):
        self.dels.append(idx)
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.data = np.delete(self.org_data,self.dels,axis=0)
        
        p = self.org_data[idx]
        
        node = self.tree.which_leaf[idx]
        ind = np.where(node.dataset == p)
        node.dataset = np.delete(node.dataset, ind[0][0],axis = 0)

        while (node.parent != None):
            for centroid in node.centroids :
                ind = np.where(node.parent.dataset == centroid)
                node.parent.dataset = np.delete(node.parent.dataset,ind[0][0],axis = 0)
            
            km = Kmeans(np.array(node.dataset),self.k,self.T)
            km.fit(np.array(node.dataset))
            node.centroids,cl,loss = km.centroids,km.clusters,km.loss
            node.parent.dataset = np.vstack([node.parent.dataset,node.centroids])
            node = node.parent
        
        km1 = Kmeans(np.array(node.dataset),self.k,self.T)
        km1.fit(np.array(node.dataset))
        node.centroids = km1.centroids
        
        km2 = Kmeans(self.data,self.k,self.T)
        km2.centroids,km2.data = node.centroids,self.data
        km2.find_clusters()
        
        self.centroids = km2.centroids
        self.clusters = km2.clusters
        self.loss = km2.loss



In [8]:
data_file = open("scaled_data.p",'rb')  
total_data = pickle.load(data_file)

In [9]:
mnist = total_data['mnist']
botattack = total_data['bot_attack']
celltype = total_data['4celltypes_10pca']
postures = total_data['postures']
covtype = total_data['covtype_multiclass']

In [10]:
def make_data(datatuple):
    X = datatuple[0]
    y = datatuple[1]
    d = X.shape[1]
    k = datatuple[2]
    print(X.shape,k)
    return X,y,d,k

In [24]:
def dis(data):
    for key in data.keys():
        print(key)
        tup = data[key]
        print(tup[0].shape,tup[1].shape,tup[2])
        print()
        
dis(total_data)

mnist
(60000, 784) (60000,) 10

bot_attack
(1018298, 115) (1018298,) 11

4celltypes_10pca
(12009, 10) (12009,) 4

postures
(74975, 15) (74975,) 5

covtype_multiclass
(15120, 52) (15120,) 7



In [11]:
X,y,d,k = make_data(covtype)

(15120, 52) 7


In [12]:
km = Kmeans(X,k,10)

st = time.time()
km.fit(X)
en = time.time()

centers,clusters,loss = km.centroids,km.clusters,km.loss

print(en-st)
print(loss)

3.7334582805633545
15233.66245694127


In [13]:
qkm = Q_Kmeans(X,k,10,0.2,0.5)

st = time.time()
qkm.qfit(X)
en = time.time()

centers,clusters,loss = qkm.centroids,qkm.clusters,qkm.loss

print(en-st)
print(loss)

1.6870388984680176
29752.184403481606


In [14]:
dckm = DC_Kmeans(X,k,10,50,1)

st = time.time()
dckm.dcfit(X)
en = time.time()

centers,clusters,loss = dckm.centroids,dckm.clusters,dckm.loss

print(en-st)
print(loss)

3.827191114425659
16266.059891619798


In [19]:
dels = random.sample(range(len(X)),1000)

In [20]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    dckm.dcdelete(dels[d]) 
en = time.time()

print(en-st)

processing deletion request 1 ...
processing deletion request 2 ...
processing deletion request 3 ...
processing deletion request 4 ...
processing deletion request 5 ...
processing deletion request 6 ...
processing deletion request 7 ...
processing deletion request 8 ...
processing deletion request 9 ...
processing deletion request 10 ...
processing deletion request 11 ...
processing deletion request 12 ...
processing deletion request 13 ...
processing deletion request 14 ...
processing deletion request 15 ...
processing deletion request 16 ...
processing deletion request 17 ...
processing deletion request 18 ...
processing deletion request 19 ...
processing deletion request 20 ...
processing deletion request 21 ...
processing deletion request 22 ...
processing deletion request 23 ...
processing deletion request 24 ...
processing deletion request 25 ...
processing deletion request 26 ...
processing deletion request 27 ...
processing deletion request 28 ...
processing deletion request 2

processing deletion request 232 ...
processing deletion request 233 ...
processing deletion request 234 ...
processing deletion request 235 ...
processing deletion request 236 ...
processing deletion request 237 ...
processing deletion request 238 ...
processing deletion request 239 ...
processing deletion request 240 ...
processing deletion request 241 ...
processing deletion request 242 ...
processing deletion request 243 ...
processing deletion request 244 ...
processing deletion request 245 ...
processing deletion request 246 ...
processing deletion request 247 ...
processing deletion request 248 ...
processing deletion request 249 ...
processing deletion request 250 ...
processing deletion request 251 ...
processing deletion request 252 ...
processing deletion request 253 ...
processing deletion request 254 ...
processing deletion request 255 ...
processing deletion request 256 ...
processing deletion request 257 ...
processing deletion request 258 ...
processing deletion request 

processing deletion request 460 ...
processing deletion request 461 ...
processing deletion request 462 ...
processing deletion request 463 ...
processing deletion request 464 ...
processing deletion request 465 ...
processing deletion request 466 ...
processing deletion request 467 ...
processing deletion request 468 ...
processing deletion request 469 ...
processing deletion request 470 ...
processing deletion request 471 ...
processing deletion request 472 ...
processing deletion request 473 ...
processing deletion request 474 ...
processing deletion request 475 ...
processing deletion request 476 ...
processing deletion request 477 ...
processing deletion request 478 ...
processing deletion request 479 ...
processing deletion request 480 ...
processing deletion request 481 ...
processing deletion request 482 ...
processing deletion request 483 ...
processing deletion request 484 ...
processing deletion request 485 ...
processing deletion request 486 ...
processing deletion request 

processing deletion request 688 ...
processing deletion request 689 ...
processing deletion request 690 ...
processing deletion request 691 ...
processing deletion request 692 ...
processing deletion request 693 ...
processing deletion request 694 ...
processing deletion request 695 ...
processing deletion request 696 ...
processing deletion request 697 ...
processing deletion request 698 ...
processing deletion request 699 ...
processing deletion request 700 ...
processing deletion request 701 ...
processing deletion request 702 ...
processing deletion request 703 ...
processing deletion request 704 ...
processing deletion request 705 ...
processing deletion request 706 ...
processing deletion request 707 ...
processing deletion request 708 ...
processing deletion request 709 ...
processing deletion request 710 ...
processing deletion request 711 ...
processing deletion request 712 ...
processing deletion request 713 ...
processing deletion request 714 ...
processing deletion request 

processing deletion request 916 ...
processing deletion request 917 ...
processing deletion request 918 ...
processing deletion request 919 ...
processing deletion request 920 ...
processing deletion request 921 ...
processing deletion request 922 ...
processing deletion request 923 ...
processing deletion request 924 ...
processing deletion request 925 ...
processing deletion request 926 ...
processing deletion request 927 ...
processing deletion request 928 ...
processing deletion request 929 ...
processing deletion request 930 ...
processing deletion request 931 ...
processing deletion request 932 ...
processing deletion request 933 ...
processing deletion request 934 ...
processing deletion request 935 ...
processing deletion request 936 ...
processing deletion request 937 ...
processing deletion request 938 ...
processing deletion request 939 ...
processing deletion request 940 ...
processing deletion request 941 ...
processing deletion request 942 ...
processing deletion request 

In [21]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    qkm.qdelete(dels[d]) 
en = time.time()

print(en-st)

processing deletion request 1 ...
processing deletion request 2 ...
processing deletion request 3 ...
processing deletion request 4 ...
processing deletion request 5 ...
processing deletion request 6 ...
processing deletion request 7 ...
processing deletion request 8 ...
processing deletion request 9 ...
processing deletion request 10 ...
processing deletion request 11 ...
processing deletion request 12 ...
processing deletion request 13 ...
processing deletion request 14 ...
processing deletion request 15 ...
processing deletion request 16 ...
processing deletion request 17 ...
processing deletion request 18 ...
processing deletion request 19 ...
processing deletion request 20 ...
processing deletion request 21 ...
processing deletion request 22 ...
processing deletion request 23 ...
processing deletion request 24 ...
processing deletion request 25 ...
processing deletion request 26 ...
processing deletion request 27 ...
processing deletion request 28 ...
processing deletion request 2

processing deletion request 232 ...
processing deletion request 233 ...
processing deletion request 234 ...
processing deletion request 235 ...
processing deletion request 236 ...
processing deletion request 237 ...
processing deletion request 238 ...
processing deletion request 239 ...
processing deletion request 240 ...
processing deletion request 241 ...
processing deletion request 242 ...
processing deletion request 243 ...
processing deletion request 244 ...
processing deletion request 245 ...
processing deletion request 246 ...
processing deletion request 247 ...
processing deletion request 248 ...
processing deletion request 249 ...
processing deletion request 250 ...
processing deletion request 251 ...
processing deletion request 252 ...
processing deletion request 253 ...
processing deletion request 254 ...
processing deletion request 255 ...
processing deletion request 256 ...
processing deletion request 257 ...
processing deletion request 258 ...
processing deletion request 

processing deletion request 460 ...
processing deletion request 461 ...
processing deletion request 462 ...
processing deletion request 463 ...
processing deletion request 464 ...
processing deletion request 465 ...
processing deletion request 466 ...
processing deletion request 467 ...
processing deletion request 468 ...
processing deletion request 469 ...
processing deletion request 470 ...
processing deletion request 471 ...
processing deletion request 472 ...
processing deletion request 473 ...
processing deletion request 474 ...
processing deletion request 475 ...
processing deletion request 476 ...
processing deletion request 477 ...
processing deletion request 478 ...
processing deletion request 479 ...
processing deletion request 480 ...
processing deletion request 481 ...
processing deletion request 482 ...
processing deletion request 483 ...
processing deletion request 484 ...
processing deletion request 485 ...
processing deletion request 486 ...
processing deletion request 

processing deletion request 688 ...
processing deletion request 689 ...
processing deletion request 690 ...
processing deletion request 691 ...
processing deletion request 692 ...
processing deletion request 693 ...
processing deletion request 694 ...
processing deletion request 695 ...
processing deletion request 696 ...
processing deletion request 697 ...
processing deletion request 698 ...
processing deletion request 699 ...
processing deletion request 700 ...
processing deletion request 701 ...
processing deletion request 702 ...
processing deletion request 703 ...
processing deletion request 704 ...
processing deletion request 705 ...
processing deletion request 706 ...
processing deletion request 707 ...
processing deletion request 708 ...
processing deletion request 709 ...
processing deletion request 710 ...
processing deletion request 711 ...
processing deletion request 712 ...
processing deletion request 713 ...
processing deletion request 714 ...
processing deletion request 

processing deletion request 916 ...
processing deletion request 917 ...
processing deletion request 918 ...
processing deletion request 919 ...
processing deletion request 920 ...
processing deletion request 921 ...
processing deletion request 922 ...
processing deletion request 923 ...
processing deletion request 924 ...
processing deletion request 925 ...
processing deletion request 926 ...
processing deletion request 927 ...
processing deletion request 928 ...
processing deletion request 929 ...
processing deletion request 930 ...
processing deletion request 931 ...
processing deletion request 932 ...
processing deletion request 933 ...
processing deletion request 934 ...
processing deletion request 935 ...
processing deletion request 936 ...
processing deletion request 937 ...
processing deletion request 938 ...
processing deletion request 939 ...
processing deletion request 940 ...
processing deletion request 941 ...
processing deletion request 942 ...
processing deletion request 

In [22]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    km.delete(dels[d])
    print(time.time()-st)
en = time.time()

print(en-st)

processing deletion request 1 ...
3.889683485031128
processing deletion request 2 ...
7.904366731643677
processing deletion request 3 ...
11.856602668762207
processing deletion request 4 ...
15.68380355834961
processing deletion request 5 ...
19.15174436569214
processing deletion request 6 ...
22.838383674621582
processing deletion request 7 ...
26.243870735168457
processing deletion request 8 ...
29.924371480941772
processing deletion request 9 ...
33.946709394454956
processing deletion request 10 ...
38.03271794319153
processing deletion request 11 ...
42.51974296569824
processing deletion request 12 ...
46.25400900840759
processing deletion request 13 ...
49.846919536590576
processing deletion request 14 ...
53.40858292579651
processing deletion request 15 ...
57.03271126747131
processing deletion request 16 ...
60.56316900253296
processing deletion request 17 ...
64.0779778957367
processing deletion request 18 ...
67.60837054252625
processing deletion request 19 ...
71.216903209686

545.8670072555542
processing deletion request 154 ...
549.3661594390869
processing deletion request 155 ...
552.7560305595398
processing deletion request 156 ...
556.1771006584167
processing deletion request 157 ...
559.6293911933899
processing deletion request 158 ...
563.0504670143127
processing deletion request 159 ...
566.4090895652771
processing deletion request 160 ...
569.8152043819427
processing deletion request 161 ...
573.3918042182922
processing deletion request 162 ...
576.8441545963287
processing deletion request 163 ...
580.4057931900024
processing deletion request 164 ...
584.1861941814423
processing deletion request 165 ...
587.7946949005127
processing deletion request 166 ...
591.4188482761383
processing deletion request 167 ...
594.8555777072906
processing deletion request 168 ...
598.6203248500824
processing deletion request 169 ...
602.1195087432861
processing deletion request 170 ...
605.4937214851379
processing deletion request 171 ...
609.0241467952728
processing

1076.5151376724243
processing deletion request 306 ...
1080.0299444198608
processing deletion request 307 ...
1083.6540687084198
processing deletion request 308 ...
1087.137630701065
processing deletion request 309 ...
1090.6680889129639
processing deletion request 310 ...
1093.9798147678375
processing deletion request 311 ...
1097.3227853775024
processing deletion request 312 ...
1100.7594830989838
processing deletion request 313 ...
1104.1336958408356
processing deletion request 314 ...
1107.5547745227814
processing deletion request 315 ...
1110.9133620262146
processing deletion request 316 ...
1114.256334066391
processing deletion request 317 ...
1117.896080493927
processing deletion request 318 ...
1121.2547013759613
processing deletion request 319 ...
1124.5508081912994
processing deletion request 320 ...
1127.9874756336212
processing deletion request 321 ...
1131.3617198467255
processing deletion request 322 ...
1134.6578245162964
processing deletion request 323 ...
1138.01641631

1588.9019966125488
processing deletion request 456 ...
1592.2293410301208
processing deletion request 457 ...
1595.7284982204437
processing deletion request 458 ...
1599.0558474063873
processing deletion request 459 ...
1602.414469718933
processing deletion request 460 ...
1605.7418196201324
processing deletion request 461 ...
1609.037925004959
processing deletion request 462 ...
1612.4589698314667
processing deletion request 463 ...
1615.755075931549
processing deletion request 464 ...
1619.0980768203735
processing deletion request 465 ...
1622.5537900924683
processing deletion request 466 ...
1626.0831623077393
processing deletion request 467 ...
1629.4974071979523
processing deletion request 468 ...
1632.7553524971008
processing deletion request 469 ...
1636.178435087204
processing deletion request 470 ...
1639.4760491847992
processing deletion request 471 ...
1642.8626866340637
processing deletion request 472 ...
1646.1971678733826
processing deletion request 473 ...
1649.709127664

2089.8405809402466
processing deletion request 606 ...
2093.0611481666565
processing deletion request 607 ...
2096.4009301662445
processing deletion request 608 ...
2099.948457479477
processing deletion request 609 ...
2103.261288166046
processing deletion request 610 ...
2106.618912935257
processing deletion request 611 ...
2109.926568508148
processing deletion request 612 ...
2113.1319563388824
processing deletion request 613 ...
2116.4643177986145
processing deletion request 614 ...
2119.651606798172
processing deletion request 615 ...
2122.9487228393555
processing deletion request 616 ...
2126.303608417511
processing deletion request 617 ...
2129.554916381836
processing deletion request 618 ...
2132.868148088455
processing deletion request 619 ...
2136.3526883125305
processing deletion request 620 ...
2139.588899374008
processing deletion request 621 ...
2142.9267580509186
processing deletion request 622 ...
2146.399376630783
processing deletion request 623 ...
2149.7438588142395
p

2591.340053319931
processing deletion request 757 ...
2594.5449397563934
processing deletion request 758 ...
2597.8470215797424
processing deletion request 759 ...
2601.3789598941803
processing deletion request 760 ...
2604.5516357421875
processing deletion request 761 ...
2607.958128452301
processing deletion request 762 ...
2611.1922628879547
processing deletion request 763 ...
2614.4425172805786
processing deletion request 764 ...
2617.5995769500732
processing deletion request 765 ...
2620.860407590866
processing deletion request 766 ...
2624.1451013088226
processing deletion request 767 ...
2627.4119675159454
processing deletion request 768 ...
2630.5866124629974
processing deletion request 769 ...
2633.790556192398
processing deletion request 770 ...
2637.041316509247
processing deletion request 771 ...
2640.5420110225677
processing deletion request 772 ...
2643.746425628662
processing deletion request 773 ...
2647.184189081192
processing deletion request 774 ...
2650.685769796371

3082.719071149826
processing deletion request 908 ...
3086.042238712311
processing deletion request 909 ...
3089.230005264282
processing deletion request 910 ...
3092.4803218841553
processing deletion request 911 ...
3095.6529693603516
processing deletion request 912 ...
3099.1370368003845
processing deletion request 913 ...
3102.3556277751923
processing deletion request 914 ...
3105.603559732437
processing deletion request 915 ...
3108.852769613266
processing deletion request 916 ...
3112.754706144333
processing deletion request 917 ...
3115.9859490394592
processing deletion request 918 ...
3119.2164013385773
processing deletion request 919 ...
3122.690820455551
processing deletion request 920 ...
3125.925982952118
processing deletion request 921 ...
3129.267858028412
processing deletion request 922 ...
3132.5792424678802
processing deletion request 923 ...
3135.876860141754
processing deletion request 924 ...
3139.1044149398804
processing deletion request 925 ...
3142.33891582489
pro